## Task:

- use Keras :)
- **Predict if employee stayed or left the company**
- build confusion matrix
- experiment with parameters to get better accuracy

Necesary data pre-processing steps:
- Data Normalization (*) 
- Spliting data into training and testing sets


(\*) Read more about Scikit's ```MinMaxScaler``` or ```Normalizer``` on:
- [Normalization](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-normalization)
- [MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)
- [Normalizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer)

----

Dataset source: https://www.kaggle.com/ludobenistant/hr-analytics

## Additional task:

Requirements: Python and maybe basic Numpy

Pretend that the dataset is too bit to fit into single Matrix. 

**Write generator that will iterate over the data during model training.**

The generator should:

- accept batch of any size
- generate batches for training/testing data and training/testing labels
- be usable by keras ```model.fit_generator``` (check method **"fit_generator"** on [Keras page](https://keras.io/models/sequential/))


In [1]:
import pandas as pd
import numpy as np

In [2]:
hr_dataset = pd.read_csv('datasets/HR_comma_sep.csv')
hr_dataset.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


# Data Exploration

Let's look at some basic features and correlations
We can just select dataset where employee left vs stayed, and look at some basic stats of the different columns. 

E.g. `satisfaction_level`, and `promotion_last_5_years` unsurprisingly seem to be quite important. 

In [3]:
# compare left vs stayed
hr_dataset[hr_dataset['left']==1].describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000,3571.0,3571.000000
mean,0.440098,0.718113,3.855503,207.419210,3.876505,0.047326,1.0,0.005321
std,0.263933,0.197673,1.818165,61.202825,0.977698,0.212364,0.0,0.072759
min,0.090000,0.450000,2.000000,126.000000,2.000000,0.000000,1.0,0.000000
25%,0.130000,0.520000,2.000000,146.000000,3.000000,0.000000,1.0,0.000000
50%,0.410000,0.790000,4.000000,224.000000,4.000000,0.000000,1.0,0.000000
75%,0.730000,0.900000,6.000000,262.000000,5.000000,0.000000,1.0,0.000000
max,0.920000,1.000000,7.000000,310.000000,6.000000,1.000000,1.0,1.000000


In [4]:
# compare left vs stayed
hr_dataset[hr_dataset['left']==0].describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.0,11428.000000
mean,0.666810,0.715473,3.786664,199.060203,3.380032,0.175009,0.0,0.026251
std,0.217104,0.162005,0.979884,45.682731,1.562348,0.379991,0.0,0.159889
min,0.120000,0.360000,2.000000,96.000000,2.000000,0.000000,0.0,0.000000
25%,0.540000,0.580000,3.000000,162.000000,2.000000,0.000000,0.0,0.000000
50%,0.690000,0.710000,4.000000,198.000000,3.000000,0.000000,0.0,0.000000
75%,0.840000,0.850000,4.000000,238.000000,4.000000,0.000000,0.0,0.000000
max,1.000000,1.000000,6.000000,287.000000,10.000000,1.000000,0.0,1.000000


# Pre-processing

We will normalize and clean up data for Keras models.


In [5]:
# one-hot encode sales
hr_dataset = hr_dataset.join(pd.get_dummies(hr_dataset['sales']), rsuffix='_sales').drop('sales', axis=1)

# one-hot encode salary
hr_dataset = hr_dataset.join(pd.get_dummies(hr_dataset['salary']), rsuffix='_salary').drop('salary', axis=1)

In [6]:
hr_dataset.shape

(14999, 21)

In [7]:
# shuffle for random
hr_dataset = hr_dataset.reindex(np.random.permutation(hr_dataset.index))

In [8]:
# split data into features and labels
labels = hr_dataset[['left']].astype(float)
feats = hr_dataset.drop('left', axis=1).astype(float)

In [9]:
# normalize data
from sklearn.preprocessing import Normalizer

n = Normalizer()
df = pd.DataFrame(n.fit_transform(feats))

In [10]:
def split_datasets(dataset, ratio=0.8):
    train_size = int(len(dataset) * ratio)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    return train, test

# split train/test
x_train, x_test = split_datasets(df.values)
y_train, y_test = split_datasets(labels.values)

In [11]:
print x_train.shape, x_test.shape
print y_train.shape, y_test.shape

# check labels in train vs test
print "train left: %s " % str(y_train.sum() / len(y_train))
print "test left: %s " % str(y_test.sum() / len(y_test))

(11999, 20) (3000, 20)
(11999, 1) (3000, 1)
train left: 0.236686390533 
test left: 0.243666666667 


In [12]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.np_utils import to_categorical


Using Theano backend.


In [13]:
model = Sequential()
model.add(Dense(20, input_dim=20))
model.add(Dense(32))
# model.add(Dropout(0.5))
# model.add(Dense(16))
# model.add(Dropout(0.5))
# model.add(Dense(8))
# model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


Since we are training for recognizing categories (i.e. left or stayed):

1) We use `categorical_crossentropy` for the loss function
2) Instead of passing in `y_train` directly, we need to one-hot encode it to [left, stayed] array. Keras has a handy function `to_categorical` we can use

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
to_categorical(y_train)

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [16]:
# train model
model.fit(x_train, to_categorical(y_train), nb_epoch=50, batch_size=4, validation_split=0.2)

Train on 9599 samples, validate on 2400 samples
Epoch 1/50
9599/9599 [==============================] - 0s - loss: 0.5423 - val_loss: 0.5544

# Predictions

How did we do?
Let's use the model to make predictions. 
Note that the predictions have 2 columns -- left, and stayed. 

We compare with `to_categorical(y_test)` to one-hot encode our test labels to 2 columns. 
Keras has a `categorical_accuracy` metric we can use to measure the error. 
Final score is 0.806 -- Its a so-so score for a Neural Network prediction.

In [17]:
# predict for test data
preds = model.predict(x_test)

In [18]:
preds

array([[ 0.96817136,  0.03182864],
       [ 0.87078327,  0.12921672],
       [ 0.72454727,  0.27545273],
       ..., 
       [ 0.52438545,  0.47561452],
       [ 0.82624406,  0.17375596],
       [ 0.7003119 ,  0.2996881 ]], dtype=float32)

In [19]:
to_categorical(y_test)

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [20]:
from keras.metrics import categorical_crossentropy, categorical_accuracy
categorical_accuracy(to_categorical(y_test), preds).eval()

array(0.8116666674613953, dtype=float32)